In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My\ Drive/cv/day29_31/Object_Detection

/content/drive/My Drive/cv/day29_31/Object_Detection


In [3]:
ls

 annotations/             Faster_RCNN_Keras/   ssd.py
'Build RetinaNet.ipynb'   images/              utils/
'Build SSD_VGG.ipynb'     layers/              輸入資料型態.ipynb
 data/                    OHEM_Focal.png
 demo/                    __pycache__/


In [4]:
"""
Ref
1.
https://blog.csdn.net/wfei101/article/details/78597442
2.
http://www.chenjianqu.com/show-92.html
3.
https://www.lizenghai.com/archives/35732.html
4.
https://hellozhaozheng.github.io/z_post/PyTorch-SSD/#MultiBox
5.
https://zhuanlan.zhihu.com/p/77736067
6.
https://www.twblogs.net/a/5c177b61bd9eee5e40bbc788
7.
https://zhuanlan.zhihu.com/p/85538108

"""

'\nRef\n1.\nhttps://blog.csdn.net/wfei101/article/details/78597442\n2.\nhttp://www.chenjianqu.com/show-92.html\n3.\nhttps://www.lizenghai.com/archives/35732.html\n4.\nhttps://hellozhaozheng.github.io/z_post/PyTorch-SSD/#MultiBox\n5.\nhttps://zhuanlan.zhihu.com/p/77736067\n6.\nhttps://www.twblogs.net/a/5c177b61bd9eee5e40bbc788\n7.\nhttps://zhuanlan.zhihu.com/p/85538108\n\n'

In [0]:
from ssd import build_ssd

## box_utils裡面很多Function，可以看看是怎麼設計的
from layers.box_utils import * 
from torch.autograd import Variable
from layers import functions
from layers import modules
from math import sqrt 
from itertools import product
from torch.autograd import Function

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import argparse
import pickle
import torch.nn.functional as F


In [6]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)

/content/drive/My Drive/cv/day29_31/Object_Detection/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [7]:
ssd_net.cfg

{'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'clip': True,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'min_dim': 300,
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'name': 'VOC',
 'num_classes': 21,
 'steps': [8, 16, 32, 64, 100, 300],
 'variance': [0.1, 0.2]}

In [0]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [9]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
# https://www.twblogs.net/a/5c177b61bd9eee5e40bbc788
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90

# 以feature map上每個點的中點爲中心（offset=0.5），生成一些列同心的prior box(然後中心點的座標會乘以step，相當於從feature map位置映射回原圖位置)
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step): #20 37 54 71 88
    ## 從min_ratio至max_ratio + 1每隔步驟(step)= 17取一個值，賦值給比(ratio)。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    # SSD 300中prior box設置並不能和paper中的公式對應
    # min_size和max_size對應到的是最小和最大的正方形邊長
    min_sizes.append(min_dim * ratio / 100.)  #[60.0,111.0,162.0,213.0,264.0]
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長(映射步長)，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  

# min_size/max_size都是用於寬高比為1的box的預測.[2]用於預測宽高比為2:1和1:2的box.
# [2,3]則預測4個shape, 1;2,2:1,1:3,3:1
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [0]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    # #cfg是定義在data/config.py裡面，是模型的相關配置
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        # 輸出特徵圖的數量
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        # 尺度
        self.feature_maps = cfg['feature_maps']
        # 計算先驗框用到的Smin和Smax
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        # 各個輸出特徵圖每個像素對應到原圖的大小
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            # product(A, repeat=4) 和 product(A, A, A, A) 是一樣的。
            # product(A, B) 和 ((x,y) for x in A for y in B) 返回結果一樣。
            # product(range(2), repeat=3) --> 000 001 010 011 100 101 110 111
            # (0 , 0) ...(0 , 37) , ... (37 , 37)
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize (/300)後再乘上8
                # 是輸出特徵圖的下採樣率，則f_k是特徵圖的邊長
                # unit center x,y
                '''中心點'''
                # 每個先驗框中心的計算公式是：(cx,cy)=((i+0.5)/|fk|, (j+0.5)/|fk|)，其中|fk|是特徵圖的邊長
                # 計算中心點,這裡的j是沿x方向變化的
                # (0.5 , 0.5) , (1.5 , 0.5) , (2.5 , 0.5)
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                """
                每個特徵圖使用的先驗框大小：
                Sk=Smin+(Smax-Smin)(k-1)/(m-1)，k的值[1,m]，m是輸出特徵圖的數量，
                Sk是先驗框相對於整張圖片的比例，Smin=0.2，Smax=0.95
                先驗框的寬度w_k_a=Sk*ar^0.5，高度h_k_a=Sk/ar^0.5，
                其中ar是特徵圖的比例，有{3, 2, 1, 1/2, 1/3}
                """
                '''/self.image_size 就是在做normalization '''
                # aspect_ratio: 1
                # rel size: min_size
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                # 相當於extend 
                # extend: Extends list by appending elements from the iterable.
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        # 此時的mean是1*34928的list，要4個數就分割出来，所以需要用view,從而變成[8732,4],即有8732个瞄點框

        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            # 對每個元素進行截斷限制,限制為[0,1]之間
            output.clamp_(max=1, min=0)
        return output

In [0]:
PriorBox_Demo=PriorBox(cfg)

In [12]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [0]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        # 交集的閾值
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        # 背景標籤, 0
        self.background_label = bkg_label
        # False
        self.encode_target = encode_target
        # True
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        # True
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        # 0.5 判定負樣本的閾值
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        # 整個prediction是剛才forward的階段的輸出
        loc_data, conf_data, priors = predictions
        # num = batch_size
        num = loc_data.size(0)
        # [8732 , 4]
        # loc_data.size(1) = 8732, 因此 priors 維持不變
        priors = priors[:loc_data.size(1), :]
        # num_priors = 8732
        num_priors = (priors.size(0))
        # num_classes = 21 (默認為voc數據集)
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            # targets是列表, 列表的長度為batch_size, 列表中每個元素為一個tensor,
            # 其shape為[num_objs, 5], 其中num_objs為當前圖片中物體的數量, 第二維前4個元素為邊框坐標, 最後一個元素為類別編號(1~20)
            # [num_objs, 4]
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            # [8732, 4]
            # .data 返回和 x 的相同數據tensor, 但不會加入到x的計算歷史裡
            defaults = priors.data

            # from ..box_utils import match
            # 關鍵函數, 實現候選框與真實框之間的匹配, 注意是候選框而不是預測結果框! 
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            # 注意! 要清楚 Python 中的參數傳遞機制, 此處在函數内部會改變 loc_t, conf_t 的值
            # threshold:確定是否匹配的交集比閾值
            # truths: (tensor: [num_obj, 4]) 存儲真實box的邊框坐標
            # priors: (tensor: [num_priors, 4], 即[8732, 4]), 存儲推薦框的坐標,注意,此時的框是default box,而不是SSD網路預測出来的框的坐標
            # 預測的結果存儲在loc_data中,其shape為[num_obj, 8732, 4].
            # 其中 num_objs 為當前圖片中物體的數量
            # idx: batches 中圖片的序號, 標示當前正在處理的image在batches中的序號
            # loc_t: (tensor: [batches, 8732, 4]),
            # conf_t: (tensor: [batches, 8732])
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        # 用Variable封裝loc_t,新版本的PyTorch無需這麼做,只需要將requires_grad屬性設置為True就行了
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        # 篩選出>0的box下標(大部分都是=0的)
        pos = conf_t > 0
        # 求和,取得滿足條件的box的數量,[batch_size, num_gt_threshold]
        num_pos = pos.sum(dim=1, keepdim=True)

        # torch.unsqueeze()這個函數主要是對數據維度進行擴充。給指定位置加上維數為一的維度
        # b.expand_as(a)就是將b進行擴充，擴充到a的維度
        # loc_data:[batch, num_priors, 4]
        # pos: [batch, num_priors]
        # pos_idx: [batch, num_priors, 4],複製下標成坐標格式, 以便獲取坐標值
        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        # 獲取預測結果值
        # view相當於reshape
        loc_p = loc_data[pos_idx].view(-1, 4)
        # 獲取gt值
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        # 計算最大的置信度, 以進行難負樣本挖掘
        # conf_data: [batch, num_priors, num_classes]
        # batch_conf: [batch, num_priors, num_classes]
        # reshape
        batch_conf = conf_data.view(-1, self.num_classes)


        # conf_t: [batch, num_priors]
        # loss_c: [batch*num_priors, 1], 計算每個priorbox預測後的損失
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        # 難負樣本挖掘, 按照loss进行排序, 取loss最大的負樣本參與更新
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        # 獲取到負樣本的下標
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [0]:
Use_cuda=False
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [15]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=False
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

/content/drive/My Drive/cv/day29_31/Object_Detection/ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


In [0]:
'''要不要使用gpu'''
Use_cuda=False

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=32
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [17]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [0]:
epochs=1
iteration=10

In [24]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        #########################################
        print(number__)
        #########################################
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
   #torch.save(ssd_net.state_dict(),'./drive/My Drive/cv/day29_31/Object_Detection/weights.pth')

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


1
2
3
4
5
6
7
8
9
BBOX Regression Loss:  2.5144689489293985
Classification Loss:  12.243745365849247


## 想要Train VOC,COCO可以參考原Github:https://github.com/amdegroot/ssd.pytorch